# Ch05. 문서 유사도(Document Similarity)

# v02. 여러 가지 유사도 기법

- 문서의 유사도를 구하기 위한 방법으로는 코사인 유사도 외에도 여러 가지 방법들이 있다.

<br>

## 2.1 유클리드 거리 (Euclidean distance)

- 유클리드 거리는 문서의 유사도를 구할 때 자카드 유사도나 코사인 유사도만큼, 유용한 방법은 아니다.
- 하지만 여러 가지 방법을 이해하고, 시도해 보는 것 자체만으로 다른 개념들을 이해할 때 도움이 되므로 의미가 있다.

<br>

### 2.1.1 유클리드 거리 공식

- 다차원 공간에서 두 개의 점 $p$와 $q$가 각각 다음과 같은 좌표를 갖는다.
  - $p = (p_1, p_2, p_3, \cdots, p_n)$
  - $q = (q_1, q_2, q_3, \cdots, q_n)$
- 이 때 두 점 사이의 거리를 계산하는 유클리드 거리 공식은 다음과 같다.

$
\qquad
\sqrt{ {\left( q_1 - p_1 \right)}^2 + {\left( q_2 - p_2 \right)}^2 + \cdots + {\left( q_n - p_n \right)}^2 } = \sqrt{ \sum_{i=1}^n {\left( q_i - p_i \right)}^2 }
$

<br>

### 2.1.2 좌표 평면 시각화

- 다차원 공간이라고 가정하면 식이 복잡하여 이해하기 어렵다.
- 좀 더 쉽게 이해하기 위해 2차원 공간이라고 가정하고 두 점 사이의 거리를 좌표 평면 상에서 시각화해보자.

<img src="../_images/Ch05/02/img001.jpg" />

- 2차원 평면 상에서 두 점 $p$와 $q$ 사이의 직선 거리를 구하는 문제이다.
- 위의 경우 직각 삼각형으로 표현 가능 $\rightarrow$ 피타고라스의 정리를 통해 $p$와 $q$ 사이의 거리를 계산할 수 있다.
- 즉, 2차원 좌표 평면에서 두 점 사이의 유클리드 거리 공식은 피타고라스의 정리를 통해 두 점 사이의 거리를 구하는 것과 동일하다.

<br>

### 2.1.3 유클리드 거리 공식을 통한 문서 유사도 구하기

- 여러 문서에 대해서 유사도를 구하고자 유클리드 거리 공식을 사용한다는 것은, 앞서 본 2차원을 단어의 총 개수만큼의 차원으로 확장하는 것과 같다.  
  

- ex) 아래와 같은 DTM이 있다고 하자

| -     | 바나나 | 사과 | 저는 | 좋아요 |
| :---- | :----- | :--- | :--- | :----- |
| 문서1 | 2      | 3    | 0    | 1      |
| 문서2 | 1      | 2    | 3    | 1      |
| 문서3 | 2      | 1    | 2    | 2      |

- 단어의 개수 = 4 $\rightarrow$ 4차원 공간에 문서1, 문서2, 문서3을 배치하는 것과 같다.  

- 이 때 다음과 같은 문서Q에 대해서 문서1, 문서2, 문서3 중 가장 유사한 문서를 찾아내고자 한다.

| -     | 바나나 | 사과 | 저는 | 좋아요 |
| :---- | :----- | :--- | :--- | :----- |
| 문서Q | 1      | 1    | 0    | 1      |

- 이 때 유클리드 거리를 통해 유사도를 구하려고 한다면, 문서Q 또한 다른 문서들처럼 4차원 공간에 배치시켰다는 관점에서 4차원 공간에서의 각각의 문서들과의 유클리드 거리를 구하면 된다.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np

def dist(x, y):
  return np.sqrt(np.sum((x-y)**2))

In [3]:
doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))

print(dist(doc1,docQ))
print(dist(doc2,docQ))
print(dist(doc3,docQ))

2.23606797749979
3.1622776601683795
2.449489742783178


- 유클리드 거리의 값이 가장 작다 $\rightarrow$ 문서 간의 거리가 가장 가깝다
- 즉, 문서1이 문서Q와 가장 유사하다고 볼 수 있다.

<br>

## 2.2 자카드 유사도 (Jaccard similarity)

### 2.2.1 자카드 유사도의 아이디어

- $A$와 $B$, 2개의 집합이 있다고 하자.
- 이 때 합집합에서 교집합의 비율을 구한다면 두 집합 $A$와 $B$의 유사도를 구할 수 있다는 것이 자카드 유사도의 아이디어이다.

<br>

### 2.2.2 자카드 유사도 특징

- 자카드 유사도는 0과 1사이의 값을 가진다.
- 만약 두 집합이 동일하다면 1의 값을 갖는다.
- 만약 두 집합의 공통 원소가 없다면 0의 값을 갖는다.

<br>

### 2.2.3 자카드 유사도 함수

- 자카드 유사도를 구하는 함수를 $J$라고 했을 때, 자카드 유사도 함수 $J$는 아래와 같다.

$
\qquad
J(A,B) = {{|A \cap B|} \over {|A \cup B|}} = { {| A \cap B |} \over {|A| + |B| - | A \cap B |} }
$

<br>

### 2.2.4 문서 유사도를 구하기 위한 자카드 유사도

- 두 개의 비교할 문서를 각각 $doc_1$, $doc_2$ 라고 했을 때 $doc_1$과 $doc_2$의 문서의 유사도를 구하기 위한 자카드 유사도는 아래와 같다.

$
\qquad
J(doc_1,doc_2) = {{doc_1 \; \cap \; doc_2} \over {doc_1 \; \cup \; doc_2}}
$

- 즉, 두 문서 $doc_1$, $doc_2$ 사이의 자카드 유사도 $J(doc_1,doc_2)$는 두 집합의 교집합 크기를 두 집합의 합집합의 크기로 나눈 값으로 정의된다.

In [4]:
# 다음과 같은 두 개의 문서가 있다.
# 두 문서 모두에 등장하는 단어는 "apple"과 "banana" 2개이다.
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화 수행
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


<br>

문서1과 문서2의 **합집합** 구하기

In [5]:
union = set(tokenized_doc1).union(set(tokenized_doc2))
print(union)

{'likey', 'holder', 'everyone', 'love', 'card', 'banana', 'coupon', 'like', 'watch', 'apple', 'you', 'passport'}


In [6]:
len(union)

12

- 문서1과 문서2의 합집합의 단어의 총 개수는 12개이다.

<br>

문서1과 문서2의 **교집합** 구하기

In [7]:
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)

{'apple', 'banana'}


In [8]:
len(intersection)

2

- 문서1과 문서2에서 둘 다 등장하는 단어는 "banana"와 "apple" 총 2개 이다.

<br>

자카드 유사도 계산

- 교집합의 수를 합집합의 수로 나누면 자카드 유사도가 계산된다.

In [10]:
print(len(intersection)/len(union))

0.16666666666666666


- 이 값이 자카드 유사도이자, 두 문서의 총 단어 집합에서 두 문서에서 공통적으로 등장한 단어의 비율이기도 하다.